In [1]:
import QuantLib as ql
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression

today = ql.Date().todaysDate()
ql.Settings.instance().evaluationDate = today

In [2]:
from src.utils.data_gathering import options_gathering, column_helper
from src.volatility.vol_surfaces import build_black_variance_surface, plot_vol_surface
from src.pricing_models.tree_model import Tree_USVanilla, get_USImpliedVol

df = options_gathering(
    "AAPL",
    expiry_start_date="2025-07-25",
    expiry_end_date="2030-12-01",
    option_type='call',
    observed_vol=0.22,
    min_strike=200,
    max_strike=250
)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,currency,Option Type,Ticker,expiration,date,time_to_maturity,underlying_price,dividend_yield,risk_free_rate,observed_vol
0,AAPL250815C00200000,2025-03-17 18:17:16+00:00,200.0,26.15,0.0,0.0,0.0,0.0,11.0,747,...,USD,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22
1,AAPL250815C00205000,2025-03-17 19:43:17+00:00,205.0,23.52,0.0,0.0,0.0,0.0,17.0,338,...,USD,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22
2,AAPL250815C00210000,2025-03-17 19:43:17+00:00,210.0,20.32,0.0,0.0,0.0,0.0,58.0,533,...,USD,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22
3,AAPL250815C00215000,2025-03-17 19:30:56+00:00,215.0,17.10,0.0,0.0,0.0,0.0,162.0,890,...,USD,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22
4,AAPL250815C00220000,2025-03-17 19:53:15+00:00,220.0,14.45,0.0,0.0,0.0,0.0,40.0,886,...,USD,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,AAPL271217C00230000,2025-03-17 19:54:59+00:00,230.0,40.40,0.0,0.0,0.0,0.0,5.0,230,...,USD,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22
97,AAPL271217C00235000,2025-03-17 17:58:54+00:00,235.0,38.40,0.0,0.0,0.0,0.0,3.0,28,...,USD,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22
98,AAPL271217C00240000,2025-03-17 18:40:30+00:00,240.0,36.00,0.0,0.0,0.0,0.0,76.0,574,...,USD,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22
99,AAPL271217C00245000,2025-03-14 15:05:11+00:00,245.0,32.76,0.0,0.0,0.0,0.0,NaN,19,...,USD,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22


In [3]:
df_cop = df.rename(columns=column_helper())
df_cop = get_USImpliedVol(df_cop)
df_cop

df filed nans for 0.00% of values


,contractSymbol,lastTradeDate,k,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,option_type,Ticker,exercise_date,date,time_to_maturity,u,div,r,sigma,calculated_vol
0,AAPL250815C00200000,2025-03-17 18:17:16+00:00,200.0,26.15,0.0,0.0,0.0,0.0,11.0,747,...,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22,0.311082
1,AAPL250815C00205000,2025-03-17 19:43:17+00:00,205.0,23.52,0.0,0.0,0.0,0.0,17.0,338,...,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22,0.317395
2,AAPL250815C00210000,2025-03-17 19:43:17+00:00,210.0,20.32,0.0,0.0,0.0,0.0,58.0,533,...,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22,0.307933
3,AAPL250815C00215000,2025-03-17 19:30:56+00:00,215.0,17.10,0.0,0.0,0.0,0.0,162.0,890,...,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22,0.294349
4,AAPL250815C00220000,2025-03-17 19:53:15+00:00,220.0,14.45,0.0,0.0,0.0,0.0,40.0,886,...,CALL,AAPL,2025-08-15,2025-03-18 14:13:58.923059,0.407940,214.0,0.004689,0.041094,0.22,0.287254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,AAPL271217C00230000,2025-03-17 19:54:59+00:00,230.0,40.40,0.0,0.0,0.0,0.0,5.0,230,...,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22,0.273451
97,AAPL271217C00235000,2025-03-17 17:58:54+00:00,235.0,38.40,0.0,0.0,0.0,0.0,3.0,28,...,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22,0.272723
98,AAPL271217C00240000,2025-03-17 18:40:30+00:00,240.0,36.00,0.0,0.0,0.0,0.0,76.0,574,...,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22,0.268523
99,AAPL271217C00245000,2025-03-14 15:05:11+00:00,245.0,32.76,0.0,0.0,0.0,0.0,NaN,19,...,CALL,AAPL,2027-12-17,2025-03-18 14:13:58.923059,2.746064,214.0,0.004689,0.041094,0.22,0.257715


In [5]:
bsv = build_black_variance_surface(df_cop, vol_col = 'calculated_vol')
vol_grid, strike_mesh, ttm_mesh = plot_vol_surface(bsv)